# Chapter 9. Decorators and Closures

Sẽ ok nếu tuân chặt theo *class-centered object orientation*. Tuy nhiên, nếu muốn implement function decorators, cẩn hiểu rõ closures và `nonlocal`

we need to cover:

- How Python evaluates decorator syntax

- How Python decides whether a variable is local

- Why closures exist and how they work

- What problem is solved by `nonlocal`

and then:

- Implementing a well-behaved decorator

- Powerful decorators in the standard library: @cache, @lru_cache, and @singledispatch

- Implementing a parameterized (tham số hoá) decorator

## Decorators 101

A *decorator* is a callable that takes another function as an argument (the decorated function).

Ví dụ:

```python
@decorate
def target():
    print('running target()')
```
có tác dụng tương tự như:
```python
def target():
    print('running target()')

target = decorate(target)
```

In [4]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [5]:
@deco
def target():
    print('running target()')

target() # run deco(target)

running inner()


In [7]:
# target is a now a reference to inner
target

<function __main__.deco.<locals>.inner()>

Sometimes call `@deco` is actually convenient, especially when doing `metaprogramming`—changing program behavior at runtime.

NOTE:
- Decorators are executed immediately when a module is loaded.

## When Python Executes Decorators

decorators chạy ngay khi decorated function được define (thường ở import time)

In [8]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

if __name__ == '__main__':
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

running register(<function f1 at 0x7fcd21720160>)
running register(<function f2 at 0x7fcd21720280>)
running main()
registry -> [<function f1 at 0x7fcd21720160>, <function f2 at 0x7fcd21720280>]
running f1()
running f2()
running f3()


`running register` được chạy đầu tiên. Và khi call @register, nó nhận decorated function object như 1 tham số (`<function f1 at 0x100631bf8>`)


--> function decorators (`@deco`) are executed as soon as the module is imported, but the decorated functions only run when they are explicitly invoked. This highlights the difference between what Pythonistas call _import time_ and _runtime_.

## Registration Decorators

Considering how decorators are commonly employed in real code:
- A real decorator is usually defined in one module and applied to functions in other modules.
- Most decorators define 1 inner function and return it.

To understand closures, we need to review how variable scopes work in Python

## Variable Scope Rules

In [9]:
b = 6 # global variable

def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

khi Python biên dịch phần thân của hàm, nó quyết định rằng `b` là một biến cục bộ vì nó được gán bên trong hàm.

Python không yêu cầu bạn khai báo biến, nhưng giả định rằng biến được gán trong phần thân của hàm là cục bộ. Khác với JavaScript khi không khai báo biến, nếu không khai báo biến cục bộ với `var` , sẽ ghi đè lên biến toàn cục.

In [10]:
b = 6

def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [11]:
b

9

Biến có thể đến từ:
- The module global scope
- The f3 function local scope
- nonlocal (is fundamental for closures)

## Closures

Closure is a function, it can access nonglobal variables that are defined outside of its body.



Ví dụ: Trung bình giá. Mỗi ngày, 1 gía mới được thêm

```Python
>>> avg(10)
10.0
>>> avg(11)
10.5
>>> avg(12)
11.0
```

Xem các cách mà avg lưu lịch sử giá và tính toán

In [14]:
# Example 9-7. average_oo.py: a class to calculate a running average
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [15]:
avg = Averager()
avg(10)

10.0

In [16]:
avg(11)

10.5

In [30]:
# Example 9-8. average.py: a higher-order function to calculate a running average

def make_averager():
    series = [] # <-- list: mutable
    # Each time an averager is called, it appends the passed argument to the series
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)

    return averager

avg = make_averager()
avg(10)

10.0

In [24]:
avg(11)

10.5

Example 9-7, `avg` is an instance of `Averager`,
- `self.series` keep history

Example 9-8, `avg` is the inner function, `averager`
- `series` is a free variable, that keep history

free variable : biến không bị ràng buộc trong phạm vi cục bộ

![img.png](9-1.png)

In [25]:
avg.__code__.co_varnames

('new_value', 'total')

The value for `series` is kept in the` __closure__` attribute of the returned function `avg`

In [27]:
avg.__closure__

(<cell at 0x7fcd216f63e0: list object at 0x7fcd21776680>,)

Each item in `avg.__closure__` corresponds to a name in `avg.__code__.co_freevars`

In [26]:
avg.__code__.co_freevars

('series',)

items are `cells`

In [28]:
avg.__closure__[0].cell_contents

[10, 11]

Tóm lại:

Closure là một hàm giữ lại các ràng buộc của các free variables tồn tại khi hàm được define, để chúng có thể được sử dụng sau này khi hàm được gọi và phạm vi xác định không còn khả dụng

## The nonlocal Declaration

Ví dụ 9-8 ko tối ưu khi mà nó luôn tính lại giá trị trung bình khi 1 biến mới được thêm vào.

In [29]:
# Example 9-12: Lỗi
def make_averager():
    count = 0 # <-- immutable type
    total = 0 # <-- immutable type

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

--> Sử dụng nonlocal

In [31]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)
avg(11)

10.5

Tóm tắt cách thức hoạt động của tính năng tra cứu biến của Python.

Dựa trên các quy tắc sau:
- `global x`, `x` is assigned to the `x` global variable module.
- `nonlocal x`, `x` is assigned to the `x` local variable of the nearest surrounding function where x is defined.
- If `x` is assigned a value in the function body, `x` is the local variable.
- If `x` is referenced but is not assigned and is not a parameter:
    - `x` will be looked up in the local scopes of the surrounding function bodies (nonlocal scopes).
    - If not found in surrounding scopes, it will be read from the module global scope.
    - If not found in the global scope, it will be read from `__builtins__.__dict__`.

## Implementing a Simple Decorator

Example 9-14. clockdeco0.py: simple decorator to show the running time of functions

In [32]:
import time

def clock(func):
    def clocked(*args): # Define inner function
        t0 = time.perf_counter()
        result = func(*args) # func free variable
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked

# 1. Records the initial time t0.
# 2. Calls the original factorial function, saving the result.
# 3. Computes the elapsed time.
# 4. Formats and displays the collected data.
# 5. Returns the result saved in step 2.

In [34]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(0.123)')
    print(snooze(0.123))
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(0.123)
[0.12638821s] snooze(0.123) -> None
None
**************************************** Calling factorial(6)
[0.00000025s] factorial(1) -> 1
[0.00000679s] factorial(2) -> 2
[0.00001108s] factorial(3) -> 6
[0.00001563s] factorial(4) -> 24
[0.00002021s] factorial(5) -> 120
[0.00002596s] factorial(6) -> 720
6! = 720


### How Above Works

```
@clock
    def factorial(n):
```

Bằng với

```
factorial = clock(factorial)
```

`clock` gets the `factorial` function as its `func` argument, then creates and returns the `clocked` function.

So `factorial` now actually holds a reference to the `clocked` function.

From now on, each time `factorial(n)` is called, `clocked(n)` gets executed.

Đây là hành vi điển hình của một `decorator`: nó thay thế hàm được `decorated` bằng một hàm mới chấp nhận cùng các đối số và (thông thường) trả về bất kỳ giá trị nào mà hàm được `decorated` lẽ ra phải trả về, đồng thời thực hiện một số xử lý bổ sung.

Kiểu như: *“Attach additional responsibilities to an object dynamically.”*

Ví dụ trên có 1 hạn chế: Nó che đi `__name__` và `__doc__` của decorated function.

--> Sử dụng `funcools.wraps` decorator để sao chép các thuộc tính có liên quan từ `func` sang `clocked`

In [35]:
import time
import functools


def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(0.123)')
    print(snooze(0.123))
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(0.123)
[0.12804767s] snooze(0.123) -> None
None
**************************************** Calling factorial(6)
[0.00000046s] factorial(1) -> 1
[0.00000742s] factorial(2) -> 2
[0.00001133s] factorial(3) -> 6
[0.00001492s] factorial(4) -> 24
[0.00001917s] factorial(5) -> 120
[0.00002954s] factorial(6) -> 720
6! = 720


See decorator `cache`
## Decorators in the Standard Library

Python has three built-in functions that are designed to decorate methods: `property` (chapter 22), `classmethod` (chapter 11), and `staticmethod` (chapter 11)

Một số decorator quan trọng:
- `functools.wraps`
- `cache`, `lru_cache`, and `singledispatch` từ module `functools`.

### Memoization with functools.cache

Từ Python 3.9
Trước Python 3.9, dùng `@lru_cache`

In [36]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)


if __name__ == '__main__':
    print(fibonacci(3))

[0.00000071s] fibonacci(1) -> 1
[0.00000071s] fibonacci(0) -> 0
[0.00000029s] fibonacci(1) -> 1
[0.00001567s] fibonacci(2) -> 1
[0.00009433s] fibonacci(3) -> 2
2


In [37]:
import functools

# @cache is applied on the function returned by @clock.
# fibonacci = cache(clock(fibonacci))
@functools.cache
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)


if __name__ == '__main__':
    print(fibonacci(3))

[0.00000079s] fibonacci(1) -> 1
[0.00000121s] fibonacci(0) -> 0
[0.00003900s] fibonacci(2) -> 1
[0.00048083s] fibonacci(3) -> 2
2


Bên cạnh việc làm cho các thuật toán đệ quy ngớ ngẩn trở nên khả thi, @cache thực sự tỏa sáng trong các ứng dụng cần fetch thông tin từ các remote API.

Tuy nhiên, `cache` ngốn bộ nhớ, phù hợp với short-lived command-line scripts.

Với long-running processes, nên dùng `functools.lru_cache`

### Using lru_cache

Ưu điểm chính của @lru_cache là mức sử dụng bộ nhớ của nó bị giới hạn bởi tham số `maxsize` (default: 128) với cơ chế LRU (Least Recently Used)

In [ ]:
import functools

# typed=True --> f(1) và f(1.0) lưu riêng biệt
@functools.lru_cache(maxsize=2**20, typed=True)
def costly_function(a, b):

### Single Dispatch Generic Functions

In [38]:
# VD: tạo hiển thị HTML cho các loại đối tượng Python khác nhau để debug

import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

In [39]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [41]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


`@singledispatch` marks the base function that handles the object type.

Xem cách dùng dưới đây

In [42]:
from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers

# @singledispatch marks the base function
# that handles the object type
@singledispatch
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

# Each specialized function is decorated
# with @«base».register
@htmlize.register
def _(text: str) -> str:
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

# Hoặc có thể add type hints cho decorated function
@htmlize.register(decimal.Decimal)
@htmlize.register(float)
@htmlize.register(fractions.Fraction)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'

Phần tiếp theo chỉ ra cách xây dựng các decorators chấp nhận các tham số ( như `@lru_cache()` và `htmlize.register(float)`)

## Parameterized Decorators

Ví dụ

In [43]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x7fcd217200d0>)
running main()
registry -> [<function f1 at 0x7fcd217200d0>]
running f1()


### A Parameterized Registration Decorator

In [44]:
# Example 9-22. To accept parameters,
# the new register decorator must be called as a function

# is now a set, so adding and removing functions is faster.
registry = set()

def register(active=True):
    def decorate(func):
        print('running register'
              f'(active={active})->decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x7fcd000477f0>)
running register(active=True)->decorate(<function f2 at 0x7fcd21720280>)


In [45]:
registry # only f2

{<function __main__.f2()>}

In [46]:
# register() expression returns decorate, which is then applied to f3.
register()(f3)

running register(active=True)->decorate(<function f3 at 0x7fcd217201f0>)


<function __main__.f3()>

In [47]:
registry # có f2 và f3

{<function __main__.f2()>, <function __main__.f3()>}

In [48]:
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x7fcd21720280>)


<function __main__.f2()>

In [49]:
registry # chỉ còn f3

{<function __main__.f3()>}

### The Parameterized Clock Decorator

Khi mà user truyền vào format string để kiểm soát đầu ra

In [50]:
# Example 9-24. Module clockdeco_param.py:
# the parameterized clock decorator

import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT): # clock is our parameterized decorator factory
    def decorate(func): # is the actual decorator.
        def clocked(*_args): # wraps the decorated function
            t0 = time.perf_counter()
            _result = func(*_args) # _args holds the actual arguments of clocked, while args is str used for display.
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals())) # **locals() here allows any local variable of clocked to be referenced
            return _result
        return clocked
    return decorate

if __name__ == '__main__':

    @clock()
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)

[0.12802854s] snooze(0.123) -> None
[0.12803387s] snooze(0.123) -> None
[0.12808967s] snooze(0.123) -> None


In [51]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12804358299763408s
snooze: 0.12519683400751092s
snooze: 0.12805595800455194s


### A Class-Based Clock Decorator

decorators are best coded as classes implementing `__call__`, and not as functions like the examples in this chapter

In [62]:
# Example 9-27. Module clockdeco_cls.py: parameterized clock decorator implemented as class

# So sánh vs Example 9-24

import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

# Instead of a clock outer function,
# the clock class is our parameterized decorator factory
class Clock:

    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt

    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked

In [70]:
@Clock()
def snooze(seconds):
    time.sleep(seconds)

snooze(.123)

[0.12803833s] snooze(0.123) -> None
